In [ ]:
! mkdir ~/.kaggle

! cp kaggle.json ~/.kaggle/

! chmod 600 ~/.kaggle/kaggle.json

mkdir: cannot create directory ‘/root/.kaggle’: File exists


In [ ]:
! pip install xgboost

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
! pip install dateparser
! pip install catboost

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 293.3/293.3 KB 15.8 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.6/76.6 MB 10.2 MB/s eta 0:00:00


In [ ]:
!kaggle datasets download chrisfilo/urbansound8k

100% 5.59G/5.61G [00:27<00:00, 223MB/s]
100% 5.61G/5.61G [00:27<00:00, 216MB/s]


In [ ]:
!unzip urbansound8k.zip

Le flux de sortie a été tronqué et ne contient que les 5000 dernières lignes.
  inflating: fold4/151877-5-1-0.wav  
  inflating: fold4/154758-5-0-0.wav  
  inflating: fold4/154758-5-0-1.wav  
  inflating: fold4/154758-5-0-10.wav  
  inflating: fold4/154758-5-0-11.wav  
  inflating: fold4/154758-5-0-12.wav  
  inflating: fold4/154758-5-0-13.wav  
  inflating: fold4/154758-5-0-14.wav  
  inflating: fold4/154758-5-0-15.wav  
  inflating: fold4/154758-5-0-16.wav  
  inflating: fold4/154758-5-0-17.wav  
  inflating: fold4/154758-5-0-18.wav  
  inflating: fold4/154758-5-0-19.wav  
  inflating: fold4/154758-5-0-2.wav  
  inflating: fold4/154758-5-0-20.wav  
  inflating: fold4/154758-5-0-21.wav  
  inflating: fold4/154758-5-0-3.wav  
  inflating: fold4/154758-5-0-4.wav  
  inflating: fold4/154758-5-0-5.wav  
  inflating: fold4/154758-5-0-6.wav  
  inflating: fold4/154758-5-0-7.wav  
  inflating: fold4/154758-5-0-8.wav  
  inflating: fold4/154758-5-0-9.wav  
  inflating: fold4/155320-3-0-0.wav 

In [ ]:
# import the required libraries

import librosa
import os
import pandas as pd
import numpy as np
from numpy import loadtxt
from matplotlib import pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, f1_score
from sklearn import svm

import xgboost as xgb
from xgboost import XGBClassifier, plot_importance

import copy

# 1 - Preprocessing the data

In [ ]:
# read and store the dataset

df = pd.read_csv("UrbanSound8K.csv")
df = df.rename(columns = {"slice_file_name": "File Name"})
df.head()

,File Name,fsID,start,end,salience,fold,classID,class
0,100032-3-0-0.wav,100032,0.0,0.317551,1,5,3,dog_bark
1,100263-2-0-117.wav,100263,58.5,62.500000,1,5,2,children_playing
2,100263-2-0-121.wav,100263,60.5,64.500000,1,5,2,children_playing
3,100263-2-0-126.wav,100263,63.0,67.000000,1,5,2,children_playing
4,100263-2-0-137.wav,100263,68.5,72.500000,1,5,2,children_playing


In [ ]:
# create the function that return the dataframe we'll input in both XGBoost and KNN classifiers

path = "/content"

def get_features(hop_length = 512):
  label_name = {
      0: "air_conditioner",
      1: "car_horn",
      2: "children_playing",
      3: "dog_bark",
      4: "drilling",
      5: "engine_idling",
      6: "gun_shot",
      7: "jackhammer",
      8: "siren",
      9: "street_music"
  }
  folds = ["fold"+str(i) for i in range(1, 11)]
  df_all = []
  for i, fold in enumerate(folds):
    print(f"Loading {fold}")
    fold_path = os.path.join(path, fold)
    fold_content = os.listdir(fold_path)
    counter = 0
    for file_name in fold_content:
      counter += 1
      if counter > 2000:
        break
      label = label_name[int(file_name.split("-")[1])]
      file_path = os.path.join(fold_path, file_name)
      ts, sr = librosa.load(file_path)
      if len(ts) < 88200:
        ts = np.append(ts, [0]*(88200-len(ts)))


      # extracting all the spectral and rhythm features as arrays for each sound
      # keeping the variance + mean, min and max values for each array => 4 features per file

      zero_crossing_rate = librosa.feature.zero_crossing_rate(y=ts, frame_length=2048, hop_length=hop_length, center=True)
      zero_crossing_rate_mean = np.mean(zero_crossing_rate)
      zero_crossing_rate_var = np.var(zero_crossing_rate)
      zero_crossing_rate_min = np.min(zero_crossing_rate)
      zero_crossing_rate_max = np.max(zero_crossing_rate)

      poly_features_1 = librosa.feature.poly_features(y=ts, sr=sr, S=None, n_fft=2048, hop_length=hop_length, win_length=None, window='hann', center=True, pad_mode='constant', order=1, freq=None)
      poly_features_1_mean = np.mean(poly_features_1)
      poly_features_1_var = np.var(poly_features_1)
      poly_features_1_min = np.min(poly_features_1)
      poly_features_1_max = np.max(poly_features_1)

      poly_features_2 = librosa.feature.poly_features(y=ts, sr=sr, S=None, n_fft=2048, hop_length=hop_length, win_length=None, window='hann', center=True, pad_mode='constant', order=2, freq=None)
      poly_features_2_mean = np.mean(poly_features_2)
      poly_features_2_var = np.var(poly_features_2)
      poly_features_2_min = np.min(poly_features_2)
      poly_features_2_max = np.max(poly_features_2)

      poly_features_3 = librosa.feature.poly_features(y=ts, sr=sr, S=None, n_fft=2048, hop_length=hop_length, win_length=None, window='hann', center=True, pad_mode='constant', order=3, freq=None)
      poly_features_3_mean = np.mean(poly_features_3)
      poly_features_3_var = np.var(poly_features_3)
      poly_features_3_min = np.min(poly_features_3)
      poly_features_3_max = np.max(poly_features_3)
      
      tonnetz = librosa.feature.tonnetz(y=ts, sr=sr, chroma=None)
      tonnetz_mean = np.mean(tonnetz)
      tonnetz_var = np.var(tonnetz)
      tonnetz_min = np.min(tonnetz)
      tonnetz_max = np.max(tonnetz)

      spectral_rolloff = librosa.feature.spectral_rolloff(y=ts, sr=sr, S=None, n_fft=2048, hop_length=512, win_length=None, window='hann', center=True, pad_mode='constant', freq=None, roll_percent=0.85)
      spectral_rolloff_mean = np.mean(spectral_rolloff)
      spectral_rolloff_var = np.var(spectral_rolloff)
      spectral_rolloff_min = np.min(spectral_rolloff)
      spectral_rolloff_max = np.max(spectral_rolloff)
      
      spectral_flatness = librosa.feature.spectral_flatness(y=ts, S=None, n_fft=2048, hop_length=512, win_length=None, window='hann', center=True, pad_mode='constant', amin=1e-10, power=2.0)
      spectral_flatness_mean = np.mean(spectral_flatness)
      spectral_flatness_var = np.var(spectral_flatness)
      spectral_flatness_min = np.min(spectral_flatness)
      spectral_flatness_max = np.max(spectral_flatness)
      
      spectral_contrast = librosa.feature.spectral_contrast(y=ts, sr=sr, S=None, n_fft=2048, hop_length=512, win_length=None, window='hann', center=True, pad_mode='constant', freq=None, fmin=200.0, n_bands=6, quantile=0.02, linear=False)
      spectral_contrast_mean = np.mean(spectral_contrast)
      spectral_contrast_var = np.var(spectral_contrast)
      spectral_contrast_min = np.min(spectral_contrast)
      spectral_contrast_max = np.max(spectral_contrast)
      
      spectral_bandwidth = librosa.feature.spectral_bandwidth(y=ts, sr=sr, S=None, n_fft=2048, hop_length=512, win_length=None, window='hann', center=True, pad_mode='constant', freq=None, centroid=None, norm=True, p=2)
      spectral_bandwidth_mean = np.mean(spectral_bandwidth)
      spectral_bandwidth_var = np.var(spectral_bandwidth)
      spectral_bandwidth_min = np.min(spectral_bandwidth)
      spectral_bandwidth_max = np.max(spectral_bandwidth)
      
      spectral_centroid = librosa.feature.spectral_centroid(y=ts, sr=sr, S=None, n_fft=2048, hop_length=512, freq=None, win_length=None, window='hann', center=True, pad_mode='constant')
      spectral_centroid_mean = np.mean(spectral_centroid)
      spectral_centroid_var = np.var(spectral_centroid)
      spectral_centroid_min = np.min(spectral_centroid)
      spectral_centroid_max = np.max(spectral_centroid)
       
      rms = librosa.feature.rms(y=ts, S=None, frame_length=2048, hop_length=512, center=True, pad_mode='constant')
      rms_mean = np.mean(rms)
      rms_var = np.var(rms)
      rms_min = np.min(rms)
      rms_max = np.max(rms)
      
      mfcc = librosa.feature.mfcc(y=ts, sr=sr, S=None, n_mfcc=20, dct_type=2, norm='ortho', lifter=0)
      mfcc_mean = np.mean(mfcc)
      mfcc_var = np.var(mfcc)
      mfcc_min = np.min(mfcc)
      mfcc_max = np.max(mfcc)
      
      chroma_cens = librosa.feature.chroma_cens(y=ts, sr=sr, C=None, hop_length=512, fmin=None, tuning=None, n_chroma=12, n_octaves=7, bins_per_octave=36, cqt_mode='full', window=None, norm=2, win_len_smooth=41, smoothing_window='hann')
      chroma_cens_mean = np.mean(chroma_cens)
      chroma_cens_var = np.var(chroma_cens)
      chroma_cens_min = np.min(chroma_cens)
      chroma_cens_max = np.max(chroma_cens)

      chroma_cqt = librosa.feature.chroma_cqt(y=ts, sr=sr, C=None, hop_length=512, fmin=None, threshold=0.0, tuning=None, n_chroma=12, n_octaves=7, window=None, bins_per_octave=36, cqt_mode='full')
      chroma_cqt_mean = np.mean(chroma_cqt)
      chroma_cqt_var = np.var(chroma_cqt)
      chroma_cqt_min = np.min(chroma_cqt)
      chroma_cqt_max = np.max(chroma_cqt)
      
      chroma_stft = librosa.feature.chroma_stft(y=ts, sr=sr, S=None, n_fft=2048, hop_length=512, win_length=None, window='hann', center=True, pad_mode='constant', tuning=None, n_chroma=12)
      chroma_stft_mean = np.mean(chroma_stft)
      chroma_stft_var = np.var(chroma_stft)
      chroma_stft_min = np.min(chroma_stft)
      chroma_stft_max = np.max(chroma_stft)
      
      melspectrogram = librosa.feature.melspectrogram(y=ts, sr=sr, hop_length=hop_length)
      melspectrogram_mean = np.mean(melspectrogram)
      melspectrogram_var = np.var(melspectrogram)
      melspectrogram_min = np.min(melspectrogram)
      melspectrogram_max = np.max(melspectrogram)

      tempogram = librosa.feature.tempogram(y=ts, sr=sr, onset_envelope=None, hop_length=512, center=True, window='hann')
      tempogram_mean = np.mean(tempogram)
      tempogram_var = np.var(tempogram)
      tempogram_min = np.min(tempogram)
      tempogram_max = np.max(tempogram)

      fourier_tempogram = librosa.feature.fourier_tempogram(y=ts, sr=sr, onset_envelope=None, hop_length=512, center=True, window='hann')
      fourier_tempogram_mean = np.mean(fourier_tempogram)
      fourier_tempogram_var = np.var(fourier_tempogram)
      fourier_tempogram_min = np.min(fourier_tempogram)
      fourier_tempogram_max = np.max(fourier_tempogram)


      # appending the dataframe with the 4 features per file

      df_all.append((file_name, i+1,
                     zero_crossing_rate_mean, zero_crossing_rate_var, zero_crossing_rate_min, zero_crossing_rate_max,
                     poly_features_1_mean, poly_features_1_var, poly_features_1_min, poly_features_1_max,
                     poly_features_2_mean, poly_features_2_var, poly_features_2_min, poly_features_2_max,
                     poly_features_3_mean, poly_features_3_var, poly_features_3_min, poly_features_3_max,
                     tonnetz_mean, tonnetz_var, tonnetz_min, tonnetz_max,
                     spectral_rolloff_mean, spectral_rolloff_var, spectral_rolloff_min, spectral_rolloff_max,
                     spectral_flatness_mean, spectral_flatness_var, spectral_flatness_min, spectral_flatness_max,
                     spectral_contrast_mean, spectral_contrast_var, spectral_contrast_min, spectral_contrast_max, 
                     spectral_bandwidth_mean, spectral_bandwidth_var, spectral_bandwidth_min, spectral_bandwidth_max, 
                     spectral_centroid_mean, spectral_centroid_var, spectral_centroid_min, spectral_centroid_max, 
                     rms_mean, rms_var, rms_min, rms_max, mfcc_mean, mfcc_var, mfcc_min, mfcc_max, 
                     chroma_cens_mean, chroma_cens_var, chroma_cens_min, chroma_cens_max, 
                     chroma_cqt_mean, chroma_cqt_var, chroma_cqt_min, chroma_cqt_max, 
                     chroma_stft_mean, chroma_stft_var, chroma_stft_min, chroma_stft_max, 
                     melspectrogram_mean, melspectrogram_var, melspectrogram_min, melspectrogram_max,
                     tempogram_mean, tempogram_var, tempogram_min, tempogram_max, 
                     fourier_tempogram_mean, fourier_tempogram_var, fourier_tempogram_min, fourier_tempogram_max,
                     label))
      
  
  # giving a name to each column of the dataframe

  df_all = pd.DataFrame(df_all, columns = ["File Name", "Fold",
                                           "Zero-Crossing Rate (mean)", "Zero-Crossing Rate (var)", "Zero-Crossing Rate (min)", "Zero-Crossing Rate (max)",
                                           "1st-Order Polynomial (mean)", "1st-Order Polynomial (var)", "1st-Order Polynomial (min)", "1st-Order Polynomial (max)",
                                           "2nd-Order Polynomial (mean)", "2nd-Order Polynomial (var)", "2nd-Order Polynomial (min)", "2nd-Order Polynomial (max)",
                                           "3rd-Order Polynomial (mean)", "3rd-Order Polynomial (var)", "3rd-Order Polynomial (min)", "3rd-Order Polynomial (max)",
                                           "Tonal Centroid (mean)", "Tonal Centroid (var)", "Tonal Centroid (min)", "Tonal Centroid (max)",
                                           "Center Frequency (mean)", "Center Frequency (var)", "Center Frequency (min)", "Center Frequency (max)",
                                           "Tonality Coefficient (mean)", "Tonality Coefficient (var)", "Tonality Coefficient (min)", "Tonality Coefficient (max)",
                                           "Spectral Contrast (mean)", "Spectral Contrast (var)", "Spectral Contrast (min)", "Spectral Contrast (max)",
                                           "Spectral Bandwidth (mean)", "Spectral Bandwidth (var)", "Spectral Bandwidth (min)", "Spectral Bandwidth (max)",
                                           "Spectral Centroid (mean)", "Spectral Centroid (var)", "Spectral Centroid (min)", "Spectral Centroid (max)",
                                           "Root-Mean-Square (mean)", "Root-Mean-Square (var)", "Root-Mean-Square (min)", "Root-Mean-Square (max)",
                                           "Mel-Frequency Cepstral Coefficients (mean)", "Mel-Frequency Cepstral Coefficients (var)", "Mel-Frequency Cepstral Coefficients (min)", "Mel-Frequency Cepstral Coefficients (max)",
                                           "Chroma Energy Normalized (mean)", "Chroma Energy Normalized (var)", "Chroma Energy Normalized (min)", "Chroma Energy Normalized (max)",
                                           "Constant-Q Chromagram (mean)", "Constant-Q Chromagram (var)", "Constant-Q Chromagram (min)", "Constant-Q Chromagram (max)",
                                           "Chromagram (mean)", "Chromagram (var)", "Chromagram (min)", "Chromagram (max)",
                                           "Melspectrogram (mean)", "Melspectrogram (var)", "Melspectrogram (min)", "Melspectrogram (max)",
                                           "Tempogram (mean)", "Tempogram (var)", "Tempogram (min)", "Tempogram (max)",
                                           "Fourier Tempogram (mean)", "Fourier Tempogram (var)", "Fourier Tempogram (min)", "Fourier Tempogram (max)",
                                           "Label"])

  return df_all

# calling the function to create the dataframe of the preprocessed data (= input for the classifiers)
df_features = get_features(690)

Loading fold1
Loading fold2
Loading fold3
Loading fold4
Loading fold5
Loading fold6
Loading fold7
Loading fold8


/usr/local/lib/python3.8/dist-packages/librosa/core/spectrum.py:222: UserWarning: n_fft=384 is too small for input signal of length=174
  warnings.warn(


Loading fold9
Loading fold10


In [ ]:
# merging the dataframe we created with the initial Kaggle dataframe to retrieve the already provided features: start, end and salience
df_model = pd.merge(df_features, df, on="File Name")

# droping the unnecessary columns
df_model = df_model.drop("File Name", axis = 1)
df_model = df_model.drop("fsID", axis = 1)
df_model = df_model.drop("fold", axis = 1)
df_model = df_model.drop("classID", axis = 1)
df_model = df_model.drop("class", axis = 1)

df_model.head()

,Fold,Zero-Crossing Rate (mean),Zero-Crossing Rate (var),Zero-Crossing Rate (min),Zero-Crossing Rate (max),1st-Order Polynomial (mean),1st-Order Polynomial (var),1st-Order Polynomial (min),1st-Order Polynomial (max),2nd-Order Polynomial (mean),...,Tempogram (min),Tempogram (max),Fourier Tempogram (mean),Fourier Tempogram (var),Fourier Tempogram (min),Fourier Tempogram (max),Label,start,end,salience
0,1,0.129105,0.000616,0.060547,0.184082,0.748055,0.698130,-0.000383,3.169619,0.677696,...,-3.881343e-16,1.0,0.639293+0.043280j,498.710663,-143.105820+11.863416j,271.920532+0.000000j,siren,6.500000,10.500000,1
1,1,0.042900,0.000032,0.017578,0.055176,2.771522,7.878515,-0.000942,7.372802,3.426228,...,-5.627068e-16,1.0,0.411665+0.002477j,172.998489,-80.8195880+0.7573860j,160.384406+0.000000j,air_conditioner,0.500000,4.500000,1
2,1,0.027588,0.000056,0.010254,0.049316,0.074437,0.005718,-0.000025,0.192266,0.096690,...,-6.276110e-16,1.0,0.465709+0.006459j,225.881454,-93.0017930+1.9575770j,183.608612+0.000000j,street_music,57.235978,61.235978,1
3,1,0.091915,0.000537,0.040039,0.154297,0.068865,0.006636,-0.000056,0.447329,0.075318,...,-4.023055e-16,1.0,0.514354+0.010440j,307.686890,-106.359688+0.000000j,204.202515+0.000000j,dog_bark,116.061785,120.061785,2
4,1,0.080112,0.000615,0.032715,0.165039,0.406111,0.167437,-0.000121,0.982066,0.510666,...,-5.542523e-16,1.0,0.454428+0.002456j,206.372925,-88.8866960+0.5245770j,176.817429+0.000000j,air_conditioner,64.000000,68.000000,2


In [ ]:
# converting the columns which do not have the correct type
df_model['Fourier Tempogram (mean)'] = df_model['Fourier Tempogram (mean)'].map(lambda x: x.real)
df_model['Fourier Tempogram (max)'] = df_model['Fourier Tempogram (max)'].map(lambda x: x.real)
df_model['Fourier Tempogram (min)'] = df_model['Fourier Tempogram (min)'].map(lambda x: x.real)

In [ ]:
df_model.dtypes

Fold                           int64
Zero-Crossing Rate (mean)    float64
Zero-Crossing Rate (var)     float64
Zero-Crossing Rate (min)     float64
Zero-Crossing Rate (max)     float64
                              ...   
Fourier Tempogram (max)      float64
Label                         object
start                        float64
end                          float64
salience                       int64
Length: 77, dtype: object

In [ ]:
# saving the dataframe to .csv to not have to run the time-consuming code each time
df_model.to_csv('df_model.csv')

# 2 - XGBoost

In [ ]:
# load the clean data set
df_model = pd.read_csv('df_model.csv', index_col = 0)

In [ ]:
df_model.head()

,Fold,Zero-Crossing Rate (mean),Zero-Crossing Rate (var),Zero-Crossing Rate (min),Zero-Crossing Rate (max),1st-Order Polynomial (mean),1st-Order Polynomial (var),1st-Order Polynomial (min),1st-Order Polynomial (max),2nd-Order Polynomial (mean),...,Tempogram (min),Tempogram (max),Fourier Tempogram (mean),Fourier Tempogram (var),Fourier Tempogram (min),Fourier Tempogram (max),Label,start,end,salience
0,1,0.129105,0.000616,0.060547,0.184082,0.748055,0.698130,-0.000383,3.169619,0.677696,...,-3.881343e-16,1.0,0.639293,498.710663,-143.105820,271.920532,siren,6.500000,10.500000,1
1,1,0.042900,0.000032,0.017578,0.055176,2.771522,7.878515,-0.000942,7.372802,3.426228,...,-5.627068e-16,1.0,0.411665,172.998489,-80.819588,160.384460,air_conditioner,0.500000,4.500000,1
2,1,0.027588,0.000056,0.010254,0.049316,0.074437,0.005718,-0.000025,0.192266,0.096690,...,-6.276110e-16,1.0,0.465709,225.881454,-93.001793,183.608612,street_music,57.235978,61.235978,1
3,1,0.091915,0.000537,0.040039,0.154297,0.068865,0.006636,-0.000056,0.447329,0.075318,...,-4.023055e-16,1.0,0.514354,307.686890,-106.359688,204.202515,dog_bark,116.061785,120.061785,2
4,1,0.080112,0.000615,0.032715,0.165039,0.406111,0.167437,-0.000121,0.982066,0.510666,...,-5.542523e-16,1.0,0.454428,206.372925,-88.886696,176.817429,air_conditioner,64.000000,68.000000,2


In [ ]:
# checking all the columns' type

with pd.option_context('display.max_rows', None):
    print(df_model.dtypes)

Fold                                            int64
Zero-Crossing Rate (mean)                     float64
Zero-Crossing Rate (var)                      float64
Zero-Crossing Rate (min)                      float64
Zero-Crossing Rate (max)                      float64
1st-Order Polynomial (mean)                   float64
1st-Order Polynomial (var)                    float64
1st-Order Polynomial (min)                    float64
1st-Order Polynomial (max)                    float64
2nd-Order Polynomial (mean)                   float64
2nd-Order Polynomial (var)                    float64
2nd-Order Polynomial (min)                    float64
2nd-Order Polynomial (max)                    float64
3rd-Order Polynomial (mean)                   float64
3rd-Order Polynomial (var)                    float64
3rd-Order Polynomial (min)                    float64
3rd-Order Polynomial (max)                    float64
Tonal Centroid (mean)                         float64
Tonal Centroid (var)        

In [ ]:
#df_model = df_model.drop(["Tempogram (min)"], axis = 1)
#df_model=df_model.dropna()

In [ ]:
# function to drop the columns that the classifiers will not use

def get_train_val(df, val_fold):
    """
        df only contains the fold, the label (as an int) and all features.
    """
    df_train = df[df["Fold"] != val_fold]
    df_val = df[df["Fold"] == val_fold]
    
    y_train = df_train["Label"]
    x_train = df_train.drop(["Label", "Fold"], axis = 1)
    
    y_val = df_val["Label"]
    x_val = df_val.drop(["Label", "Fold"], axis = 1)
    
    return x_train, x_val, y_train, y_val

In [ ]:
# function that performs cross-validation on the 10 files successively, without mixing the whole dataset
# testing the different hyperparameters of the XGBoost classifier by groups of 4 (fixing the parameters once optimized)

def cv_xgb(df):
    """
        df only contains the fold, the label (as an int) and all features.
    """
    num_folds = 10

    #hyperparameters choices (this was that of the last training)
    
    learning_rate = [0.5, 0.7, 0.9]
    reg_lambda = [2, 3, 4]
    min_child_weight = [4, 5, 6]
    subsample = [0.01, 0.5, 1]
    
    scores = []


    for lr in learning_rate:
        for rl in reg_lambda:
            for min in min_child_weight:
                for sub in subsample:

                    print(f"Training with: learning_rate: {lr}, reg_lambda: {rl}, min_child_weight: {min}, subsample: {sub}")
                   
                    list_scores = []
                   
                    for val_fold in range(1,num_folds+1):
                      
                        x_train, x_val, y_train, y_val = get_train_val(df, val_fold)

                        # XGBoost implementation
                        model = XGBClassifier(seed = 42, 
                                          learning_rate = lr, 
                                          reg_lambda = rl, 
                                          min_child_weight = min, 
                                          subsample = sub, 
                                          colsample_bytree = 1,
                                          eta = 0,
                                          gamma = 0,
                                          max_depth = 11,
                                          max_delta_step = 3.5,
                                          alpha = 0,
                                          scale_pos_weight = 0,
                                          max_leaves = 1,
                                          max_bin = 2,
                                          tree_method='gpu_hist')
                        
                        res = model.fit(x_train, y_train, 
                              early_stopping_rounds = 10, 
                              eval_set = [(x_val, y_val)], 
                              verbose = True)
                        
                        list_scores.append(res.best_score)
                      
                    scores.append(((lr, rl, min, sub), np.mean(list_scores)))
    return scores

In [34]:
# function that performs cross-validation on the 10 files successively, without mixing the whole dataset
# testing the different hyperparameters of the XGBoost classifier by groups of 4 (fixing the parameters once optimized)

def cv_xgb(df):
    """
        df only contains the fold, the label (as an int) and all features.
    """
    num_folds = 10

    #hyperparameters choices (this was that of the last training)
    
    alpha = [0, 0.5, 1]
    scale_pos_weight = [0.001, 1, 2]
    min_child_weight = [0, 1, 2]
    subsample = [2, 256, 300]
    
    scores = []


    for lr in alpha:
        for rl in scale_pos_weight:
            for min in max_leaves:
                for sub in max_bin:

                    print(f"Training with: alpha: {lr}, scale_pos_weight: {rl}, max_leaves: {min}, max_bin: {sub}")
                   
                    list_scores = []
                   
                    for val_fold in range(1,num_folds+1):
                      
                        x_train, x_val, y_train, y_val = get_train_val(df, val_fold)

                        # XGBoost implementation
                        model = XGBClassifier(seed = 42, 
                                          learning_rate = 0.3, 
                                          reg_lambda = 3, 
                                          min_child_weight = 0, 
                                          subsample = 0.5, 
                                          colsample_bytree = 1,
                                          gamma = 0,
                                          max_depth = 6,
                                          max_delta_step = 0,
                                          alpha = lr,
                                          scale_pos_weight = rl,
                                          max_leaves = min,
                                          max_bin = sub,
                                          tree_method='gpu_hist')
                        
                        res = model.fit(x_train, y_train, 
                              early_stopping_rounds = 10, 
                              eval_set = [(x_val, y_val)], 
                              verbose = True)
                        
                        list_scores.append(res.best_score)
                      
                    scores.append(((lr, rl, min, sub), np.mean(list_scores)))
    return scores

In [35]:
xgb = cv_xgb(df_model)

Training with: learning_rate: 0, reg_lambda: 0, min_child_weight: 0, subsample: 250
[0]	validation_0-merror:0.423826
Will train until validation_0-merror hasn't improved in 10 rounds.
[1]	validation_0-merror:0.398625
[2]	validation_0-merror:0.394044
[3]	validation_0-merror:0.406644
[4]	validation_0-merror:0.408935
[5]	validation_0-merror:0.400916
[6]	validation_0-merror:0.399771
[7]	validation_0-merror:0.386025
[8]	validation_0-merror:0.375716
[9]	validation_0-merror:0.383734
[10]	validation_0-merror:0.371134
[11]	validation_0-merror:0.358534
[12]	validation_0-merror:0.359679
[13]	validation_0-merror:0.359679
[14]	validation_0-merror:0.367698
[15]	validation_0-merror:0.378007
[16]	validation_0-merror:0.373425
[17]	validation_0-merror:0.367698
[18]	validation_0-merror:0.366552
[19]	validation_0-merror:0.357388
[20]	validation_0-merror:0.352806
[21]	validation_0-merror:0.350515
[22]	validation_0-merror:0.350515
[23]	validation_0-merror:0.34937
[24]	validation_0-merror:0.352806
[25]	valid

In [36]:
# returning the best parameter combination (the one that gives the smallest error)
res = sorted(xgb, key = lambda x : x[1],reverse=False)
res

[((0, 0, 0, 256), 0.3287115),
 ((0, 0.001, 0, 256), 0.3287115),
 ((0, 0, 0, 250), 0.33244830000000003),
 ((0, 0.001, 0, 250), 0.33244830000000003),
 ((0, 0, 0, 275), 0.33527609999999997),
 ((0, 0.001, 0, 275), 0.33527609999999997)]

# 3 - KNN

In [ ]:
# loading the clean data set
df_model = pd.read_csv('df_model.csv', index_col = 0)

In [ ]:
#copying the original dataset to applicate the KNN model
df_knn = df_model.copy()

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neighbors import BallTree
from sklearn.neighbors import DistanceMetric
from sklearn.neighbors import VALID_METRICS
import copy


def cv_knn(df):
    """
        df only contains the fold, the label (as an int) and all features.
    """
    num_folds = 10

    # hyperparameters choices
    n_neighbors = np.arange(20,30)
    weights = ['uniform', 'distance']
    algorithm = ['ball_tree', 'kd_tree', 'brute']
    distance=['braycurtis','canberra','chebyshev','cityblock','correlation','cosine','euclidean','jensenshannon','minkowski']
    p=1
    scores = [] # list for the result
    for n in n_neighbors:
      for w in weights : 
        print(f"Training with: n_neighbors: {n}, weights: {w}")
        list_scores = []
        for val_fold in range(1,num_folds+1):
          x_train, x_val, y_train, y_val = get_train_val(df, val_fold)  
          # KNN model implementation  
          model = KNeighborsClassifier(n_neighbors = n, weights = w,p=p,leaf_size =30, algorithm ='ball_tree',metric='canberra')
          model_fold = copy.deepcopy(model)

          res = model_fold.fit(x_train, y_train)
          list_scores.append(res.score(x_val,y_val))
    scores.append(((n,w,p,'ball_tree','canberra'), np.mean(list_scores)))
    
    return scores

In [ ]:
# application of cross validation on our data set 
knn = cv_knn(df_knn)

Training with: n_neighbors: 20, weights: uniform
Training with: n_neighbors: 20, weights: distance
Training with: n_neighbors: 21, weights: uniform
Training with: n_neighbors: 21, weights: distance
Training with: n_neighbors: 22, weights: uniform
Training with: n_neighbors: 22, weights: distance
Training with: n_neighbors: 23, weights: uniform
Training with: n_neighbors: 23, weights: distance
Training with: n_neighbors: 24, weights: uniform
Training with: n_neighbors: 24, weights: distance
Training with: n_neighbors: 25, weights: uniform
Training with: n_neighbors: 25, weights: distance
Training with: n_neighbors: 26, weights: uniform
Training with: n_neighbors: 26, weights: distance
Training with: n_neighbors: 27, weights: uniform
Training with: n_neighbors: 27, weights: distance
Training with: n_neighbors: 28, weights: uniform
Training with: n_neighbors: 28, weights: distance
Training with: n_neighbors: 29, weights: uniform
Training with: n_neighbors: 29, weights: distance


In [ ]:
# returning the best accuracy score with ((n_neighbors, weight, p, algorithm, metric), accuracy)
knn.sort(key=lambda x: x[1], reverse=True)
knn

[((29, 'distance', 1, 'ball_tree', 'canberra'), 0.5326704199394586)]